# DAC correction for Terschelling

In [1]:
import os
import xarray as xr

In [3]:
# this cell takes a few minutes
tg_lat = 53.420034
tg_lon = 5.351231

datapath = '/media/bene/Seagate/PhD-data/11_IB_DAC/DAC/'
i = 0
for root, dirs, files in os.walk(datapath):
    for fname in files:
        if fname.endswith('.nc'):
            # first dataset, to be concatenated with the following ones
            if i == 0:
                data_all = xr.open_dataset(os.path.join(root, fname))
                data_tg_all = data_all.interp(latitude=tg_lat, longitude=tg_lon, method='linear')
                # move date from attribute to dimension and coordinate
                data_tg_all_exp = data_tg_all.expand_dims(dim={'time':[data_tg_all.dac.date]}, axis=0)
                del data_tg_all_exp.dac.attrs['date'], data_tg_all_exp.dac.attrs['Date_CNES_JD']
                i = i + 1
            else:
                # data to be concatenated with data_all
                data_new = xr.open_dataset(os.path.join(root, fname))
                data_tg_new = data_new.interp(latitude=tg_lat, longitude=tg_lon, method='linear')
                # move date from attribute to dimension and coordinate
                data_tg_new_exp = data_tg_new.expand_dims(dim={'time':[data_new.dac.date]}, axis=0)
                del data_tg_new_exp.dac.attrs['date'], data_tg_new_exp.dac.attrs['Date_CNES_JD']

                data_tg_all_exp = xr.concat([data_tg_all_exp, data_tg_new_exp], dim='time')
                i = i + 1
                        
data_tg_all_exp

<xarray.Dataset>
Dimensions:    (time: 44648)
Coordinates:
  * time       (time) object '1992-12-31 18:00:00.000000 UTC' ... '2023-01-01...
    latitude   float64 53.42
    longitude  float64 5.351
Data variables:
    dac        (time) float64 -0.06648 -0.1053 -0.1419 ... 0.05784 0.1841 0.3126
Attributes:
    Conventions:  CF-1.4
    title:        MOG2D-G HR barotropic model output (high frequencies) combi...
    institution:  CNES/CNRS-LEGOS/CLS
    source:       MOG2D-G(Pressure+Wind)+Inverted Barometer, system version 2...
    history:      no
    references:   http://www.aviso.oceanobs.com
    comment:      Low Frequencies/High Frequencies:20 day cutoff

In [4]:
# save
data_tg_all_exp.to_netcdf(datapath + 'dac_terschelling.nc')